In [1]:
import time
import os
import subprocess
import sys
sys.path.insert(0,'..')
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_atac as aformer
#import src.aformer_TF as aformer
from src.layers.layers import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_atac as training_utils


from scipy import stats

2023-04-24 01:55:27.895278: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 01:55:28.082927: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-24 01:55:28.082962: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-24 01:55:29.000305: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-1')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    #options.experimental_threading.max_intra_op_parallelism = 1
    mixed_precision.set_global_policy('mixed_bfloat16')
    tf.config.optimizer.set_jit(True)
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

2023-04-24 01:55:30.384409: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-24 01:55:30.384462: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-24 01:55:30.384489: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


In [ ]:
#with strategy.scope():

model = aformer.aformer(kernel_transformation='relu_kernel_transformation',
                        dropout_rate=0.30,
                        pointwise_dropout_rate=0.10,
                        input_length=262144,
                        output_length=2048,
                        final_output_length=1536,
                        num_heads=8,
                        numerical_stabilizer=0.0000001,
                        nb_random_features=256,
                        max_seq_length=2048,
                        rel_pos_bins=2048,
                        norm=True,
                        BN_momentum=0.90,
                        use_rot_emb = True,
                        use_mask_pos = False,
                        normalize = True,
                        num_transformer_layers=6,
                        inits_type="enformer_performer",
                        load_init=False,
                        stable_variant=False,
                        freeze_conv_layers=False,
                        filter_list_seq=[768,896,1024,1152,1280,1536],
                        filter_list_atac=[32,64],
                        output_heads=["human","mouse","rhesus","rat"],
                        learnable_PE=True)


test = tf.ones((1,262144,4)),tf.ones((1,65536,1))#,tf.ones((1,1,1536))
model(test,training=False)
print('ran test input')
#model.load_weights("gs://picard-testing-176520/genformer_atac_pretrain/models/aformer_hg_mm_rm_rat_262k_load-True_LR-0.01_T-6_D-0.3_2023-04-18_12:37:42/iteration_24/saved_model")
#model.load_weights("gs://picard-testing-176520/genformer_atac_pretrain/models/aformer_hg_262k_load-True_LR-0.01_T-7_D-0.3_2023-04-13_15:39:43/final/saved_model")
model.load_weights("gs://picard-testing-176520/genformer_atac_pretrain/models/aformer_hg_262k_load-True_LR-0.01_T-6_D-0.3_2023-04-23_23:48:45/iteration_3/saved_model")
print('loaded weights')

In [ ]:
def predict_on_batch(model, inputs):
    return model.predict_on_batch(inputs)

from deeplift import dinuc_shuffle
@tf.function
def contribution_input_grad(model, model_inputs, gradient_mask):
    seq, atac =model_inputs
    
    #seq_shuffled = dinuc_shuffle.dinuc_shuffle(sequence_one_hot, 1)[0]
    
    gradient_mask = tf.cast(gradient_mask,dtype=tf.float32)
    gradient_mask_mass = tf.reduce_sum(gradient_mask)

    with tf.GradientTape() as input_grad_tape:
        input_grad_tape.watch(seq)
        input_grad_tape.watch(atac)
        #input_grad_tape.watch(tf_acc)
        inputs = seq,atac#,tf_acc
        prediction, peaks_pred, embedding, att_matrices = model.predict_on_batch(inputs)
        
        prediction = tf.cast(prediction["human"],dtype=tf.float32)
        gradient_mask = tf.cast(gradient_mask,dtype=tf.float32)
        
        prediction_mask = tf.reduce_sum(gradient_mask *
                                        prediction) / gradient_mask_mass
        

    input_grads = input_grad_tape.gradient(prediction_mask, inputs)

    input_grads_seq = input_grads[0] 
    input_grads_atac = input_grads[1]

    seq_grads = tf.reduce_sum(input_grads_seq[0,:,:] * seq[0,:,:],
                              axis=1)

    atac_grads = input_grads_atac[0,:,] * atac[0,:,]

    return seq_grads, atac_grads, input_grads_seq[0,:,:], \
            prediction, att_matrices, embedding



In [ ]:
def one_hot(sequence):
    '''
    convert input string tensor to one hot encoded
    will replace all N character with 0 0 0 0
    '''
    vocabulary = tf.constant(['A', 'C', 'G', 'T'])
    mapping = tf.constant([0, 1, 2, 3])

    init = tf.lookup.KeyValueTensorInitializer(keys=vocabulary,
                                               values=mapping)
    table = tf.lookup.StaticHashTable(init, default_value=0)

    input_characters = tfs.upper(tfs.unicode_split(sequence, 'UTF-8'))

    out = tf.one_hot(table.lookup(input_characters), 
                      depth = 4, 
                      dtype=tf.float32)
    return out


g=tf.random.Generator.from_seed(datetime.now().timestamp())
def deserialize_val(serialized_example,
                   input_length,
                   max_shift,
                   output_length_ATAC,
                   output_length,
                   crop_size,
                   output_res,
                   atac_mask,
                   log_atac,
                   g):
    """Deserialize bytes stored in TFRecordFile."""
    ## parse out feature map
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'atac': tf.io.FixedLenFeature([], tf.string),
        'tss_tokens': tf.io.FixedLenFeature([], tf.string),
        'peaks': tf.io.FixedLenFeature([], tf.string)
    }
    ### stochastic sequence shift and gaussian noise

    seq_shift=5
    stupid_random_seed = g.uniform([], 0, 10000000,dtype=tf.int32)
    input_seq_length = input_length + max_shift
    
    ## now parse out the actual data
    data = tf.io.parse_example(serialized_example, feature_map)
    sequence = one_hot(tf.strings.substr(data['sequence'],
                                 seq_shift,input_length))
    atac = tf.ensure_shape(tf.io.parse_tensor(data['atac'],
                                              out_type=tf.float32),
                           [output_length_ATAC,1])
    peaks = tf.ensure_shape(tf.io.parse_tensor(data['peaks'],
                                              out_type=tf.int32),
                           [output_length])
    peaks_crop = tf.slice(tf.expand_dims(peaks,axis=1),
                     [crop_size,0],
                     [output_length-2*crop_size,-1])
    
    atac_target = atac ## store the target

    masked_atac = atac * atac_mask
    
    random_shuffled_tokens= tf.random.experimental.stateless_shuffle(masked_atac, 
                                                                      seed=[1, stupid_random_seed])
    full_comb_mask = (1.0-atac_mask)*random_shuffled_tokens
    masked_atac = masked_atac + full_comb_mask
    
    if log_atac: 
        masked_atac = tf.math.log1p(masked_atac)
        
    diff = tf.math.sqrt(tf.nn.relu(masked_atac - 100.0 * tf.ones(masked_atac.shape)))
    masked_atac = tf.clip_by_value(masked_atac, clip_value_min=0.0, clip_value_max=100.0) + diff
        
    atac_out = tf.reduce_sum(tf.reshape(atac_target, [-1,32]),axis=1,keepdims=True)

    atac_out = tf.cast(tf.cast(atac_out,dtype=tf.float16),dtype=tf.float32) ### round to be consistent with Enformer
    diff = tf.math.sqrt(tf.nn.relu(atac_out - 2500.0 * tf.ones(atac_out.shape)))
    atac_out = tf.clip_by_value(atac_out, clip_value_min=0.0, clip_value_max=2500.0) + diff
    atac_out = tf.slice(atac_out,
                        [crop_size,0],
                        [output_length-2*crop_size,-1])

        
    tss_tokens = tf.ensure_shape(tf.io.parse_tensor(data['tss_tokens'],
                                              out_type=tf.int32),
                           [output_length-2*crop_size])
    tss_tokens=tf.expand_dims(tss_tokens,axis=1)
        
        
    return {'sequence': tf.ensure_shape(sequence,
                                        [input_length,4]),
            'atac': tf.ensure_shape(masked_atac,
                                    [output_length_ATAC,1]),
            'tss_tokens': tf.ensure_shape(tss_tokens,
                                      [output_length-crop_size*2,1]),
            'peaks': tf.ensure_shape(peaks_crop,
                                      [output_length-crop_size*2,1]),
            'target': tf.ensure_shape(atac_out,
                                      [output_length-crop_size*2,1])}

In [ ]:
atac_mask = np.ones((2048,1))
for k in tf.range(1022,1024):
    atac_mask[k,0] = 0.0
#for k in tf.range(1582,1598):
#    atac_mask[k,0] = 0.0

atac_mask = tf.constant(atac_mask,dtype=tf.float32)

atac_mask = tf.reshape(tf.tile(atac_mask, [1,32]),[-1])
atac_mask = tf.expand_dims(atac_mask,axis=1)

filenames = ["gs://picard-testing-176520/genformer_atac_pretrain/262k/genformer_atac_pretrain_globalacc_conv_rpgc_SINGLE/HG_Jurkat.chr12-9764927-9764966.CD69_promoter.tfr",
             "gs://picard-testing-176520/genformer_atac_pretrain/262k/genformer_atac_pretrain_globalacc_conv_rpgc_SINGLE/HG_Jurkat_stim.chr12-9764927-9764966.CD69_promoter.tfr"]
             
    
seqs_out = []
seq_grads_out = []
atacs_out = []
predictions_out = []
targets_out = []
full_grads_out = []
for file in filenames:
    g = tf.random.Generator.from_seed(datetime.now().timestamp())
    dataset = tf.data.TFRecordDataset(file,
                                      compression_type='ZLIB',
                                      num_parallel_reads=4)
    dataset = dataset.map(lambda record: deserialize_val(record,
                                                         262144,
                                                         10,
                                                             65536,
                                                         2048,
                                                         256,
                                                         128,
                                                         atac_mask,
                                                            True,g),
                          deterministic=False,
                          num_parallel_calls=4)
    dataset=dataset.batch(1)
    iter_test = iter(dataset)
    test = next(iter_test)
    inputs = test['sequence'], \
                test['atac']#, \
                    #test['global_acc']

    mask = np.zeros((1,1536,1))
    for k in range(1536):
        if k in range(766,769):
            mask[0,k,0]=1

    seq_grads, atac_grads,full_grads, prediction,att_matrices,embedding = contribution_input_grad(model,inputs, mask)
    
    atacs_out.append(test['atac'])
    predictions_out.append(prediction)
    targets_out.append(test['target'])
    seq_grads_out.append(seq_grads)
    full_grads_out.append(full_grads)
    
def plot_tracks(tracks, peaks, start, end, height=1.5):
    fig, axes = plt.subplots(len(tracks)+1, 1, figsize=(24, height * (len(tracks)+1)), sharex=True)
    for ax, (title, y) in zip(axes, tracks.items()):
        ax.fill_between(np.linspace(start, end, num=len(y[0])), y[0],color=y[1])
        ax.set_title(title)
    axes[-1].imshow(peaks, aspect = "auto", cmap="viridis")
    plt.tight_layout()

#for idx, atacs in enumerate(atacs_out):
    
tracks={'prediction': (predictions_out[0][0,:,0],'blue'),
        'atac_in': (tf.reduce_sum(tf.reshape((tf.math.exp(atacs_out[0][0,:,0])-1.0), [2048,32]),axis=1)[256:-256],'red'),
        'target': (targets_out[0][0,:,0],'orange'),
        'prediction1': (predictions_out[1][0,:,0],'blue'),
        'atac_in1': (tf.reduce_sum(tf.reshape((tf.math.exp(atacs_out[1][0,:,0])-1.0), [2048,32]),axis=1)[256:-256],'red'),
        'target1': (targets_out[1][0,:,0],'orange')}
        

atac_mask_expanded = (1.0 - tf.transpose(tf.slice(tf.expand_dims(tf.reshape(tf.tile(atac_mask,[1,4]),[-1]),axis=1),
         [32768,0],[196608,-1])))
plot_tracks(tracks, atac_mask_expanded,0,196608)


In [ ]:
atac_mask = np.ones((2048,1))
for k in tf.range(1022,1024):
    atac_mask[k,0] = 0.0
#for k in tf.range(1582,1598):
#    atac_mask[k,0] = 0.0

atac_mask = tf.constant(atac_mask,dtype=tf.float32)

atac_mask = tf.reshape(tf.tile(atac_mask, [1,32]),[-1])
atac_mask = tf.expand_dims(atac_mask,axis=1)

filenames = ["gs://picard-testing-176520/genformer_atac_pretrain/262k/genformer_atac_pretrain_globalacc_conv_rpgc_SINGLE/HG_Jurkat.chr12-9764927-9764966.CD69_promoter.tfr",
             "gs://picard-testing-176520/genformer_atac_pretrain/262k/genformer_atac_pretrain_globalacc_conv_rpgc_SINGLE/HG_Jurkat_stim.chr12-9764927-9764966.CD69_promoter.tfr"]
             
    
seqs_out = []
seq_grads_out = []
atacs_out = []
predictions_out = []
targets_out = []
full_grads_out = []
for file in filenames:
    g = tf.random.Generator.from_seed(datetime.now().timestamp())
    dataset = tf.data.TFRecordDataset(file,
                                      compression_type='ZLIB',
                                      num_parallel_reads=4)
    dataset = dataset.map(lambda record: deserialize_val(record,
                                                         262144,
                                                         10,
                                                             65536,
                                                         2048,
                                                         256,
                                                         128,
                                                         atac_mask,
                                                            True,g),
                          deterministic=False,
                          num_parallel_calls=4)
    dataset=dataset.batch(1)
    iter_test = iter(dataset)
    test = next(iter_test)
    inputs = test['sequence'], \
                test['atac']#, \
                    #test['global_acc']

    mask = np.zeros((1,1536,1))
    for k in range(1536):
        if k in range(766,769):
            mask[0,k,0]=1

    seq_grads, atac_grads,full_grads, prediction,att_matrices,embedding = contribution_input_grad(model,inputs, mask)
    
    atacs_out.append(test['atac'])
    predictions_out.append(prediction)
    targets_out.append(test['target'])
    seq_grads_out.append(seq_grads)
    full_grads_out.append(full_grads)
    
def plot_tracks(tracks, peaks, start, end, height=1.5):
    fig, axes = plt.subplots(len(tracks)+1, 1, figsize=(24, height * (len(tracks)+1)), sharex=True)
    for ax, (title, y) in zip(axes, tracks.items()):
        ax.fill_between(np.linspace(start, end, num=len(y[0])), y[0],color=y[1])
        ax.set_title(title)
    axes[-1].imshow(peaks, aspect = "auto", cmap="viridis")
    plt.tight_layout()

#for idx, atacs in enumerate(atacs_out):
    
tracks={'prediction': (predictions_out[0][0,:,0],'blue'),
        'atac_in': (tf.reduce_sum(tf.reshape((tf.math.exp(atacs_out[0][0,:,0])-1.0), [2048,32]),axis=1)[256:-256],'red'),
        'target': (targets_out[0][0,:,0],'orange'),
        #'prediction1': (predictions_out[1][0,:,0],'blue'),
        #'atac_in1': (tf.reduce_sum(tf.reshape(atacs_out[1][0,:,0], [2048,32]),axis=1)[256:-256],'red'),
        #'target1': (targets_out[1][0,:,0],'orange'),
        'prediction2': (predictions_out[1][0,:,0],'blue'),
        'atac_in2': (tf.reduce_sum(tf.reshape((tf.math.exp(atacs_out[1][0,:,0])-1.0), [2048,32]),axis=1)[256:-256],'red'),
        'target2': (targets_out[1][0,:,0],'orange')}
        

atac_mask_expanded = (1.0 - tf.transpose(tf.slice(tf.expand_dims(tf.reshape(tf.tile(atac_mask,[1,4]),[-1]),axis=1),
         [32768,0],[196608,-1])))
plot_tracks(tracks, atac_mask_expanded,0,196608)